In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

from scipy.spatial import Voronoi, voronoi_plot_2d
import shapely

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:25: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


### Etapa 1: Resolver para un caso simplificado

In [65]:
# Caso simplificado (Unknowns en paréntesis)

# FILAS (Manzanas)
# P_0A2_tot | P_0A2_F | P_0A2_M
#     4     |    4    |    0
#    (7)    |    5    |    2
#     3     |   (1)   |    2
#     4     |    3    |   (1)
#    (10)   |   (5)   |   (5)

# TOTALES (AGEB)
# P_0A2_tot | P_0A2_F | P_0A2_M
#     28    |   18    | 10

In [63]:
blocks_values_simplified = pd.DataFrame( {'P_0A2_tot': [4, None, 3, 4,None],
                                          'P_0A2_F': [4, 5, None, 3,None],
                                          'P_0A2_M': [0, 2, 2, None,None]})
blocks_values_simplified

,P_0A2_tot,P_0A2_F,P_0A2_M
0,4.0,4.0,0.0
1,NaN,5.0,2.0
2,3.0,NaN,2.0
3,4.0,3.0,NaN
4,NaN,NaN,NaN


In [64]:
ageb_values_simplified = pd.DataFrame( {'P_0A2_tot': [28],
                                        'P_0A2_F': [18],
                                        'P_0A2_M': [10]})
ageb_values_simplified

,P_0A2_tot,P_0A2_F,P_0A2_M
0,28,18,10


In [44]:
def row_computable_equation(args):
    
    # Extract row_vars from tupple (renaming them as eq_vars since nans will be replaced by unknowns in the format of x[i])
    eq_vars_tupple = args[1:-1]
    eq_vars = []
    for eq_var in eq_vars_tupple:
        eq_vars.append(eq_var)

    if extended_logs:
        print(f"Row {index} Step 3: Re-extracted eq_vars: {eq_vars}.")
    
    # Extract starting nan_i from tupple
    nan_i = args[-1]
    if extended_logs:
        print(f"Row {index} Step 3: Starting row with nan_i: {nan_i}.")
    
    # Find where there are nan values
    nans_index, = np.where(np.isnan(eq_vars))

    current_equation = 'P_0A2_tot - (P_0A2_F + P_0A2_M)'
    fixed_equation = current_equation

    # Extract variables from equation
    variables = current_equation
    delimiters = [" ", "+", "-", "(",")"]
    for delimiter in delimiters:
        variables = " ".join(variables.split(delimiter))
    variables = variables.split()
    
    # Replace nans with x[i]
    j = 0

    # Get global variable constrained_cols, a dictionary containing each nan_i found in each column.
    global constrained_cols

    # For each variable position
    for index in range(len(eq_vars)):
        
        # Current variable being examined in current row
        nan_col = variables[index]
        
        # If variable position was detected as nan
        if index in nans_index:

            # Replace variable name in equation with unknown variable.
            fixed_equation = fixed_equation.replace(nan_col,f'x[{nan_i}]')

            # Append current nan_i to corresponding column in constrained_cols
            try:
                col_nans = constrained_cols[nan_col]
                col_nans.append(nan_i)
                constrained_cols[nan_col] = col_nans
            except:
                constrained_cols[nan_col] = [nan_i]
            
            # Increment found nans (i)
            nan_i += 1

            j = j+1

        # If variable position was not detected as nan
        else:
            # Replace variable name in equation with known value
            fixed_equation = fixed_equation.replace(nan_col, f"{eq_vars[index]}")

    print(f"Row {index} Step 3: Created computable row equation: {fixed_equation} by replacing {j} nans as x[i] unknowns.")
    
    if extended_logs:    
        print(f"Row {index} Step 3: Finishing row with nan_i: {nan_i}.")

    return eval(fixed_equation)

In [45]:
nan_i = 0
x = [99,99,99,99,99,99]
extended_logs = False

constrained_cols = {}

def create_row_constraints(blocks_values_simplified, nan_i):

    # EXPLICACIÓN DE LAS CONSTRAINTS: Las constraints se agregan en la función minimize como lista de diccionarios.
    # En cada diccionario de restricciones, 'type':'eq' significa que la restricción es de tipo igualdad. 
    # Esto indica que queremos que una función de igualdad (definida en 'fun') sea igual a cero.
    # La función de igualdad que se debe colocar en 'fun' se crea en las siguientes definiciones. Lo que va después del return debe ser igual a cero.

    variables = ['P_0A2_tot','P_0A2_F','P_0A2_M']
    row_constraints = []
    
    #----- RESTRICCIONES QUE SON POR FILA ---------------------------------------------------------------------------------------------------------------------
    # En este caso, la restricción debería asegurar que por fila (por .iterrows) P_0A2 sea igual a P_0A2_F + P_0A2_M.
    # El número de variables encontradas (i) corresponderá al número de nans que hay en cada row.
    
    for index, row in blocks_values_simplified.iterrows():

        # STEP 1: Get row values - Obtains current row values (known values and unknown variables nans)
        # STEP 2: Set constraints args (x, val1, val2, valn, nan_i) (x is global variable, array, vals are row values, nan_i is a nan counter)
        # STEP 3: Create equation (nans get substituded as x[nan_i], computable variables)
        # STEP 4: return constraint (dicctionary type)
        
        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 1: Get row values ------------------------------------------------------------------------------------------------------------------------------
        row_vars = []
        for col in variables:
            row_vars.append(row[col])
        # Print analysis logs
        if extended_logs:
            print(f"Row {index} Step 1: Row variables: {row_vars}.")

        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 2: Set row args (val1, val2, valn, nan_i) ----------------------------------------------------------------------------------------------------------
        # Includes nans, but what's most important here is placing the non-nan-vars in the correct arg position and adding current row's starting nan_i.
        # [For example, if row_vars = [5,nan,3] and it is the first nan encountered (nan_i=0), args = (x,5,nan,3,0)]
        # Args tupple is used in a function, so this step ensures all values are in its corresponding positional argument.

        # First argument of args_list is always x, an array of unknown values used in scipy.optimize.minimize.
        args_list = [x]
        # Second arguments are current row_vars
        for var in row_vars:
            args_list.append(var)
        # Third and last argument is nan_i, which counts unknown values.
        args_list.append(nan_i)
        # Finally, turn current row's args into tuple.
        args = tuple(args_list)
        # Print analysis logs
        if extended_logs:
            print(f"Row {index} Step 2: Constraints dicc args (x, row_vars, i): {args}.")

        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 3: Create equation -----------------------------------------------------------------------------------------------------------------------------            
        fixed_equation = row_computable_equation(args)
        # Update nan_i with nans that will be substituded inside current_row_eq_args()
        nans_found = len(np.where(np.isnan(row_vars))[0])
        nan_i = nan_i + nans_found
        # Print analysis logs
        if extended_logs:
            print(f"Row {index} Step 3: Evaluated equation: {fixed_equation}.")
        
        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 4: Return constraint ---------------------------------------------------------------------------------------------------------------------------
        current_row_dicc = {'type': 'eq', 
                            'fun': row_computable_equation, 
                            'args': args}
        row_constraints.append(current_row_dicc)
        # Print analysis logs
        if extended_logs:
            print(f"Row {index} Step 4: Constraints dicc: {current_row_dicc}.")
            
    return row_constraints, nan_i

In [57]:
row_constraints, nan_i = create_row_constraints(blocks_values_simplified, nan_i=0)
print(row_constraints)
print(nan_i)

Row 2 Step 3: Created computable row equation: 4.0 - (4.0 + 0.0) by replacing 0 nans as x[i] unknowns.
Row 2 Step 3: Created computable row equation: x[0] - (5.0 + 2.0) by replacing 1 nans as x[i] unknowns.
Row 2 Step 3: Created computable row equation: 3.0 - (x[1] + 2.0) by replacing 1 nans as x[i] unknowns.
Row 2 Step 3: Created computable row equation: 4.0 - (3.0 + x[2]) by replacing 1 nans as x[i] unknowns.
Row 2 Step 3: Created computable row equation: x[3] - (x[4] + x[5]) by replacing 3 nans as x[i] unknowns.
[{'type': 'eq', 'fun': <function row_computable_equation at 0x7f075fb1d160>, 'args': ([99, 99, 99, 99, 99, 99], 4.0, 4.0, 0.0, 0)}, {'type': 'eq', 'fun': <function row_computable_equation at 0x7f075fb1d160>, 'args': ([99, 99, 99, 99, 99, 99], nan, 5.0, 2.0, 0)}, {'type': 'eq', 'fun': <function row_computable_equation at 0x7f075fb1d160>, 'args': ([99, 99, 99, 99, 99, 99], 3.0, nan, 2.0, 1)}, {'type': 'eq', 'fun': <function row_computable_equation at 0x7f075fb1d160>, 'args': (

In [90]:
extended_logs = True
def create_col_constraints(blocks_values_simplified, ageb_values_simplified, constrained_cols, nan_i):
    #----- RESTRICCIONES QUE SON POR COLUMNA  -----------------------------------------------------------------------------------------------------------------
    # En este caso, la restricción debería asegurar que por fila (por .iterrows) P_0A2 sea igual a P_0A2_F + P_0A2_M.
    # El número de variables encontradas (i) corresponderá al número de nans que hay en cada row.
    # Restricción para P_0A2
    
    col_constraints = []
    for col in constrained_cols.keys():
        
        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 1: Get col value -------------------------------------------------------------------------------------------------------------------------------
        col_val = ageb_values_simplified[col].unique()[0]
        # Print analysis logs
        if extended_logs:
            print(f"Column {col} Step 1: AGEB total value: {col_val}.")
            
        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 2: Set col args (blocks_values, unknown_col_vars, total_value) ---------------------------------------------------------------------------------
        
        # First argument of args_list is always x, an array of unknown values used in scipy.optimize.minimize.
        args_list = [x]
        # Second argument of args_list is blocks_values_simplified gdf
        args_list.append(blocks_values_simplified)
        # Third arguments are current unique unknowns (as nan_i indexes)
        unknowns = list(set(constrained_cols[col]))
        args_list.append(unknowns)
        # Fourth argument is current col name
        args_list.append(col)
        # Fourth argument is total value for current col
        args_list.append(col_val)
        
        # Finally, turn current row's args into tuple.
        args = tuple(args_list)
        # Print analysis logs
        if extended_logs:
            print(f"Column {col} Step 2: Constraints dicc args (x, blocks, unknowns, col_name, col_value): {args}.")
        
        def col_computable_equation(args):
                
            #Restricción: La suma de los valores actuales en la columna del gdf de manzanas + los valores encontrados en x hasta ahora - el valor total conocido por AGEB debe ser igual a 0.
            
            # Extract nan_i as global variable (to change global, not local value)
            x = args[0]
            blocks_values_simplified = args[1]
            unknowns_indexes = args[2]
            col_name = args[3]
            col_val = args[4]

            print(f"Reading function for col {col_name}.") #Verification log
            return np.nansum(blocks_values_simplified[col_name]) + np.nansum(x[unknowns_indexes]) - col_val

        col_constraints.append({'type': 'eq', 
                                'fun': col_computable_equation,
                                'args':(args)})

    return col_constraints
            
    a = """
    def constraint_total_P_0A2(x,blocks_values_simplified,P_0A2_unknown_vars,total_values):
        #Restricción: La suma de los valores actuales en la columna + los valores encontrados - el valor total conocido[1] debe ser igual a 0.
        #print(f"Reading function for col P_0A2.") #Verification log
        return np.nansum(blocks_relevant_numeric['P_0A2']) + np.nansum(x[P_0A2_unknown_vars]) - total_values[0] 
    constraints.append({'type': 'eq', 
                        'fun': constraint_total_P_0A2,
                        'args':(blocks_relevant_numeric,P_0A2_unknown_vars,total_values)})
    
    # Restricción para P_0A2_F
    def constraint_total_P_0A2_F(x,blocks_values_simplified,P_0A2_F_unknown_vars,total_values):
        #Restricción: La suma de los valores actuales en la columna + los valores encontrados - el valor total conocido[2] debe ser igual a 0.
        #print(f"Reading function for col P_0A2_F.") #Verification log
        return np.nansum(blocks_relevant_numeric['P_0A2_F']) + np.nansum(x[P_0A2_F_unknown_vars]) - total_values[1]
    constraints.append({'type': 'eq', 
                        'fun': constraint_total_P_0A2_F,
                        'args':(blocks_relevant_numeric,P_0A2_F_unknown_vars,total_values)})
    
    # Restricción para P_0A2_M
    def constraint_total_P_0A2_M(x,blocks_values_simplified,P_0A2_M_unknown_vars,total_values):
        #Restricción: La suma de los valores actuales en la columna + los valores encontrados - el valor total conocido[3] debe ser igual a 0.
        #print(f"Reading function for col P_0A2_M.") #Verification log
        return np.nansum(blocks_relevant_numeric['P_0A2_M']) + np.nansum(x[P_0A2_M_unknown_vars]) - total_values[2]
    constraints.append({'type': 'eq', 
                        'fun': constraint_total_P_0A2_M,
                        'args':(blocks_relevant_numeric,P_0A2_M_unknown_vars,total_values)})
                        """


col_constraints = create_col_constraints(blocks_values_simplified, ageb_values_simplified, constrained_cols, nan_i)
col_constraints

Column P_0A2_tot Step 1: AGEB total value: 28.
Column P_0A2_tot Step 2: Constraints dicc args (x, blocks, unknowns, col_name, col_value): ([99, 99, 99, 99, 99, 99],    P_0A2_tot  P_0A2_F  P_0A2_M
0        4.0      4.0      0.0
1        NaN      5.0      2.0
2        3.0      NaN      2.0
3        4.0      3.0      NaN
4        NaN      NaN      NaN, [0, 3, 6], 'P_0A2_tot', 28).
Column P_0A2_F Step 1: AGEB total value: 18.
Column P_0A2_F Step 2: Constraints dicc args (x, blocks, unknowns, col_name, col_value): ([99, 99, 99, 99, 99, 99],    P_0A2_tot  P_0A2_F  P_0A2_M
0        4.0      4.0      0.0
1        NaN      5.0      2.0
2        3.0      NaN      2.0
3        4.0      3.0      NaN
4        NaN      NaN      NaN, [1, 4], 'P_0A2_F', 18).
Column P_0A2_M Step 1: AGEB total value: 10.
Column P_0A2_M Step 2: Constraints dicc args (x, blocks, unknowns, col_name, col_value): ([99, 99, 99, 99, 99, 99],    P_0A2_tot  P_0A2_F  P_0A2_M
0        4.0      4.0      0.0
1        NaN      5.0   

[{'type': 'eq',
  'fun': <function __main__.create_col_constraints.<locals>.col_computable_equation(args)>,
  'args': ([99, 99, 99, 99, 99, 99],
      P_0A2_tot  P_0A2_F  P_0A2_M
   0        4.0      4.0      0.0
   1        NaN      5.0      2.0
   2        3.0      NaN      2.0
   3        4.0      3.0      NaN
   4        NaN      NaN      NaN,
   [0, 3, 6],
   'P_0A2_tot',
   28)},
 {'type': 'eq',
  'fun': <function __main__.create_col_constraints.<locals>.col_computable_equation(args)>,
  'args': ([99, 99, 99, 99, 99, 99],
      P_0A2_tot  P_0A2_F  P_0A2_M
   0        4.0      4.0      0.0
   1        NaN      5.0      2.0
   2        3.0      NaN      2.0
   3        4.0      3.0      NaN
   4        NaN      NaN      NaN,
   [1, 4],
   'P_0A2_F',
   18)},
 {'type': 'eq',
  'fun': <function __main__.create_col_constraints.<locals>.col_computable_equation(args)>,
  'args': ([99, 99, 99, 99, 99, 99],
      P_0A2_tot  P_0A2_F  P_0A2_M
   0        4.0      4.0      0.0
   1        Na

In [80]:
test = [0, 3, 6, 0, 3]
test = list(set(test))
test

[0, 3, 6]

In [68]:
constrained_cols
for col in constrained_cols.keys():
    print(col)

P_0A2_tot
P_0A2_F
P_0A2_M
